In [1]:
# Needs 2 nd verison
from pyspark import SparkConf
# path='/FileStore/tables/dice_com_job_us_sample-3431c.csv'
path='/FileStore/tables/Newest_Dice.csv'

# path='/FileStore/tables/Newly_Dice.csv'

# Getting the data into spark dataframe
# Using header = true to skip the header and read the data properly
df=sqlContext.read.format("csv")\
        .option("header","true")\
        .option("inferSchmea","true")\
        .load(path)


# df = sqlContext.read.format("csv")(path, inferSchema = True, header=True)
print df.dtypes
df.show(5)


In [2]:
df.describe()

In [3]:
    # Basic Stopwords
    stopwords = ['a', u'about', u'above', u'after', u'again', u'against', u'all', u'am', u'an', u'and', u'any', u'are', u'arent', u'as', u'at', 
    u'be', u'because', u'been', u'before', u'being', u'below', u'between', u'both', u'but', u'by', 
    u'can', 'cant', 'come', u'could', 'couldnt', u'd', u'did', u'didn', u'do', u'does', u'doesnt', u'doing', u'dont', u'down', u'during', 
    u'each', u'few', 'finally', u'for', u'from', u'further', 
    u'had', u'hadnt', u'has', u'hasnt', u'have', u'havent', u'having', u'he', u'her', u'here', u'hers', u'herself', u'him', u'himself', u'his', u'how', 
    u'i', u'if', u'in', u'into', u'is', u'isnt', u'it', u'its', u'itself', 
    u'just', u'll',u'm', u'me', u'might', u'more', u'most', u'must', u'my', u'myself', 
    u'no', u'nor', u'not', u'now',u'o', u'of', u'off', u'on', u'once', u'only', u'or', u'other', u'our', u'ours', u'ourselves', u'out', u'over', u'own', 
    u'r', u're', u's', 'said', u'same', u'she', u'should', u'shouldnt', u'so', u'some', u'such', 
    u't', u'than', u'that', 'thats', u'the', u'their', u'theirs', u'them', u'themselves', u'then', u'there', u'these', u'they', u'this', u'those', u'through', u'to', u'too', u'under', u'until', u'up', u'very', u'was', u'wasnt', u'we', u'were', u'werent', u'what', u'when', u'where', u'which', u'while', u'who', u'whom', u'why', u'will', u'with', u'wont', u'would', u'y', u'you', u'your', u'yours', u'yourself', u'yourselves']


In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.functions import struct

def data_cleansing(record):
    text  = record[3]
    words = text.split()
    text_out = [word.lower() for word in words if word.lower() not in stopwords]
    return text_out

udf_dataCleansing = udf(data_cleansing , ArrayType(StringType()))

processed_text = df.withColumn("words", udf_dataCleansing(struct([df[x] for x in df.columns])))

In [5]:
print(processed_text)
print(df)

In [6]:
from pyspark.ml.feature import IDF
# importing inverse document frequency
from pyspark.ml.feature import CountVectorizer
# Getting all the counts of all the words using the library CountVectorizer

cv = CountVectorizer(inputCol="words", outputCol="rawFeatures", vocabSize = 1000)
cvmodel = cv.fit(processed_text)

vectorData = cvmodel.transform(processed_text)

vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)
 
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(vectorData)
rescaledData = idfModel.transform(vectorData)
# show the transformed data
rescaledData.select('jobdescription','words','rawFeatures','features').show(10)

In [7]:
from pyspark.ml.clustering import LDA

# The value of k is essentially getting the top 10 trending topics, you can adjust your k accordingly
lda = LDA(k=10, seed=123, optimizer="em", featuresCol="features")
mymodel = lda.fit(rescaledData)
 
ldatopics = mymodel.describeTopics()
ldatopics.show(15)

In [8]:
# generate topic summary
trending_topics = list()
num = 6
# how many terms you want
for row in ldatopics.rdd.map(lambda x: x).collect():
  topic_id = [row.topic]
  indices = row.termIndices[:num]
  topic_weights = row.termWeights
  terms = [vocab[idx] for idx in indices]
  trending_topics.append( topic_id + terms )

for topic in trending_topics:
  print "Trensing job postings number " + str(topic[0]+1) + "  Terms: " + str(topic[1:])